In [ ]:
from snowflake.snowpark.context import get_active_session
import pandas as pd

session = get_active_session()

In [ ]:
def get_reconciliation_by_variable(
    session, 
    table_alias, 
    variable_name, 
    display_name,
    top_n=15,
    sort_by= None #['coupa_only_pct', 'total_count']  # List of fields to sort by
):
        # Handle sort_by parameter - convert to list if string, default to list if None
    if sort_by is None:
        sort_by = ['coupa_only_pct', 'total_count']  # Default sorting
    elif isinstance(sort_by, str):
        sort_by = [sort_by]  # Convert single string to list
    elif not isinstance(sort_by, list):
        sort_by = ['coupa_only_pct', 'total_count']  # Fallback to default
    
    # Build ORDER BY clause from the sort_by list
    order_by_clause = ', '.join([f"{field} DESC" for field in sort_by])

    query = f"""
    WITH invoice_indicator AS (
        SELECT 
            {table_alias}.{variable_name} AS variable_value, 
            CASE 
                WHEN C.INVOICE__ IS NOT NULL THEN 'BOTH'
                ELSE 'COUPA ONLY'
            END AS missing_invoice_indicator,
            COUNT(1) AS count
        FROM 
            CLEANSED.COUPA.COUPA_INVOICE_HEADER_BCV A
        JOIN
            CLEANSED.COUPA.coupa_supplier_bcv B
            ON A.SUPPLIER_ID = B.ID
        LEFT JOIN _sandbox_working_capital.working_capital_uploads.wc_ap_zdp_december C
            ON A.INVOICE_NUMBER = C.INVOICE__
        LEFT JOIN CLEANSED.COUPA.COUPA_SUPPLIER_INFORMATION_BCV d
            ON b.ID = d.supplier_id
        WHERE DATE(A.PAYMENT_DATE) BETWEEN '2025-12-01' AND '2025-12-31'
        GROUP BY 1, 2
    )

    SELECT 
        variable_value,
        COALESCE(SUM(CASE WHEN missing_invoice_indicator = 'BOTH' THEN count END), 0) AS both_count,
        COALESCE(SUM(CASE WHEN missing_invoice_indicator = 'COUPA ONLY' THEN count END), 0) AS coupa_only_count,
        SUM(count) AS total_count,
        ROUND(COALESCE(SUM(CASE WHEN missing_invoice_indicator = 'BOTH' THEN count END), 0) * 100.0 / SUM(count), 2) AS both_pct,
        ROUND(COALESCE(SUM(CASE WHEN missing_invoice_indicator = 'COUPA ONLY' THEN count END), 0) * 100.0 / SUM(count), 2) AS coupa_only_pct
    FROM invoice_indicator
    GROUP BY variable_value
    ORDER BY {order_by_clause} 
    LIMIT {top_n}
    """
    
    snowpark_df = session.sql(query)
    df = snowpark_df.to_pandas()
    df['table_alias'] = table_alias
    df['variable_name'] = variable_name
    df['display_name'] = display_name
    return df


In [ ]:
variables_to_test = [
    ('B', 'shipping_term_id', 'Supplier: Shipping Term'),
    ('B', 'payment_term_id', 'Supplier: Payment Term'),
    ('B', 'send_invoices_to_approvals', 'Supplier: Send Invoices to Approvals'),
    ('B', 'po_method', 'Supplier: PO Method'),
    ('A', 'account_type_id', 'Invoice: Account Type'),
    ('A', 'status', 'Invoice: Status'),
    ('A', 'channel', 'Invoice: Channel'),
    ('A', 'payment_channel', 'Invoice: Payment Channel'),
    ('B','name','Supplier: Supplier Name'),
    ('A','account_type_id','Invoice :Buyer'),
    ('A','requested_by_id','Invoice Requester'),
    ('A','created_by_id','Invoice : Owner'),
    ('A','currency_id','Currency'),#e.g. Zendesk Br, USA, Mx, etc.
    # 1. Test if "Missing Tax ID" causes the exclusion
    ('D', 'FEDERAL_TAX_NUM', 'Supplier: Fed Tax ID'),
    
    # 2. Test if "Missing ERP Number" causes the exclusion (CRITICAL CHECK)
    ('D', 'SUPPLIER_NUMBER', 'Supplier: Number'),
    
    # 3. Test if "Supplier Tier" causes the exclusion (EXTRACTED FROM JSON)
    # This extracts the 'name' field from the 'supplier-category' lookup inside the JSON
    ('D', 'CUSTOM_FIELDS:"supplier-category":lookup:name::STRING', 'Supplier: Tier (Custom)')
]

In [ ]:
all_results = []

for table_alias, field_name, display_name in variables_to_test:
    print(f"Processing: {display_name} ({table_alias}.{field_name})")
    try:
        #df = get_reconciliation_by_variable(session, table_alias, field_name, display_name, sort_by=['coupa_only_pct','total_count']
        df = get_reconciliation_by_variable(session, table_alias, field_name, display_name, sort_by='total_count'                                     
)
        all_results.append(df)
        print(f"  ✅ Found {len(df)} categories")
    except Exception as e:
        print(f"  ❌ Error: {e}")

# Combine all results
if all_results:
    combined_df = pd.concat(all_results, ignore_index=True)
    print(f"\n📊 Total rows: {len(combined_df)}")
else:
    print("No results collected")

# %%
# View results
combined_df

In [ ]:
def get_reconciliation_with_amounts(
    session, 
    table_alias, 
    variable_name, 
    display_name,
    top_n=20,
    sort_by=None # ['coupa_only_pct', 'total_count', 'total_sum_usd']
):
    # --- 1. Handle Sort Logic ---
    if sort_by is None:
        sort_by = ['coupa_only_pct', 'total_count']
    elif isinstance(sort_by, str):
        sort_by = [sort_by]
    elif not isinstance(sort_by, list):
        sort_by = ['coupa_only_pct', 'total_count']
    
    order_by_clause = ', '.join([f"{field} DESC" for field in sort_by])

    # --- 2. Build the Query with Scalable Rates ---
    query = f"""
    WITH rates_raw AS (
        SELECT
            TO_CURRENCY_ID,
            RATE,
            -- Create the shared "Month Bucket" key
            DATE_TRUNC('month', DATE(CREATED_AT)) as RATE_MONTH,
            ROW_NUMBER() OVER (
                PARTITION BY TO_CURRENCY_ID, DATE_TRUNC('month', DATE(CREATED_AT))
                ORDER BY CREATED_AT DESC
            ) AS rn
        FROM CLEANSED.COUPA.COUPA_EXCHANGE_RATE_BCV
        WHERE FROM_CURRENCY_ID = 1 -- Assuming 1 is USD
          AND DATE(CREATED_AT) >= '2024-01-01' -- Ensure we have history
    ),
    monthly_rates AS (
        SELECT
            TO_CURRENCY_ID,
            RATE,
            RATE_MONTH
        FROM rates_raw
        WHERE rn = 1
    ),
    invoice_details_calculated AS (
        SELECT 
            {table_alias}.{variable_name} AS variable_value, 
            CASE 
                WHEN C.INVOICE__ IS NOT NULL THEN 'BOTH'
                ELSE 'COUPA ONLY'
            END AS missing_invoice_indicator,
            -- Calculate USD Amount at the row level
            (A.GROSS_TOTAL / COALESCE(R.RATE, 1)) AS amount_usd
        FROM 
            CLEANSED.COUPA.COUPA_INVOICE_HEADER_BCV A
        -- Join Scalable Rates
        LEFT JOIN monthly_rates R
            ON A.CURRENCY_ID = R.TO_CURRENCY_ID
            AND DATE_TRUNC('month', DATE(A.PAYMENT_DATE)) = R.RATE_MONTH
        JOIN
            CLEANSED.COUPA.coupa_supplier_bcv B
            ON A.SUPPLIER_ID = B.ID
        LEFT JOIN CLEANSED.COUPA.COUPA_SUPPLIER_INFORMATION_BCV D
            ON B.ID = D.SUPPLIER_ID
        LEFT JOIN _sandbox_working_capital.working_capital_uploads.wc_ap_zdp_december C
            ON A.INVOICE_NUMBER = C.INVOICE__
        WHERE DATE(A.PAYMENT_DATE) BETWEEN '2025-12-01' AND '2025-12-31'
    ),
    grouped_stats AS (
        SELECT
            variable_value,
            missing_invoice_indicator,
            COUNT(1) AS count,
            SUM(amount_usd) AS sum_usd
        FROM invoice_details_calculated
        GROUP BY 1, 2
    )

    SELECT 
        variable_value,
        -- Counts
        COALESCE(SUM(CASE WHEN missing_invoice_indicator = 'BOTH' THEN count END), 0) AS both_count,
        COALESCE(SUM(CASE WHEN missing_invoice_indicator = 'COUPA ONLY' THEN count END), 0) AS coupa_only_count,
        SUM(count) AS total_count,
        
        -- USD Amounts (Converted to K and Rounded to 1 decimal)
        ROUND(COALESCE(SUM(CASE WHEN missing_invoice_indicator = 'BOTH' THEN sum_usd END), 0) / 1000, 1) AS both_total_usd_k,
        ROUND(COALESCE(SUM(CASE WHEN missing_invoice_indicator = 'COUPA ONLY' THEN sum_usd END), 0) / 1000, 1) AS coupa_only_total_usd_k,
        ROUND(SUM(sum_usd) / 1000, 1) AS total_usd_k,

        -- Percentages
        ROUND(COALESCE(SUM(CASE WHEN missing_invoice_indicator = 'BOTH' THEN count END), 0) * 100.0 / SUM(count), 2) AS both_pct,
        ROUND(COALESCE(SUM(CASE WHEN missing_invoice_indicator = 'COUPA ONLY' THEN count END), 0) * 100.0 / SUM(count), 2) AS coupa_only_pct
    FROM grouped_stats
    GROUP BY variable_value
    ORDER BY {order_by_clause} 
    LIMIT {top_n}
    """
    
    snowpark_df = session.sql(query)
    df = snowpark_df.to_pandas()
    df['table_alias'] = table_alias
    df['variable_name'] = variable_name
    df['display_name'] = display_name
    return df

In [ ]:
all_results = []

# Ensure variables_to_test is defined before running this!
# Example: variables_to_test = [('B', 'payment_term_id', 'Payment Terms'), ('B', 'country_id', 'Country')]

for table_alias, field_name, display_name in variables_to_test:
    print(f"Processing: {display_name} ({table_alias}.{field_name})")
    try:
        df = get_reconciliation_with_amounts(
            session, 
            table_alias, 
            field_name, 
            display_name, 
            sort_by='coupa_only_total_usd_k' # You can change this to 'total_usd' if you want to see highest value first
        )
        all_results.append(df)
        print(f"  ✅ Found {len(df)} categories")
    except Exception as e:
        print(f"  ❌ Error: {e}")

# Combine all results
if all_results:
    combined_df_amount = pd.concat(all_results, ignore_index=True)
    print(f"\n📊 Total rows: {len(combined_df_amount)}") # <--- FIXED TYPO HERE
else:
    print("No results collected")

# --- Optional: Format the output to make USD easier to read ---
# This just formatting for display, it doesn't change the underlying data
pd.options.display.float_format = '{:,.2f}'.format 

# View results
combined_df_amount

In [ ]:
combined_df_amount